# Lab 10: Fully Connected Neural Networks

In this assignment, we will learn fully connected neural network.


## 1. Example

This assignement should be run on Google Colab where you can use free GPU to accelerate the computation. Please refer to our slides to set up GPU.

### 1. Install Pytorch

In [38]:
!pip3 install torch torchvision   # install pytorch

### 2. Check GPU

In [39]:
!/opt/bin/nvidia-smi  #show GPU

Tue Jun 11 18:10:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0              26W /  70W |    169MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 3. Mount to google drive (optional)

In [40]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 4. Code

In [41]:
#Import Libraries
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [42]:
args={}
args['batch_size']=100
args['test_batch_size']=100
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset.
args['lr']=0.01 #Learning rate is how fast it will decend.
args['log_interval']=10

In [43]:
# build an mlp
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)   # linear layer (784 -> 256)
        self.fc2 = nn.Linear(256,128)  # linear layer (256 -> 128)
        self.fc3 = nn.Linear(128,10)  # linear layer (128 -> 10)

    def forward(self, x):
        h0 = x.view(-1,28*28) #input layer
        h1 = F.relu(self.fc1(h0)) # hidden layer 1
        h2 = F.relu(self.fc2(h1)) # hidden layer 2
        h3 = self.fc3(h2) # output layer

        return h3

model = Net()
model.cuda() # put the model on GPU
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [44]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = args['lr'])

In [45]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=False)

In [46]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()

        output = model(data)
        loss = criterion(output, target)

        # compute gradients
        optimizer.zero_grad()
        loss.backward()

        #to do a one-step update on our parameter.
        optimizer.step()

        #Print out the loss periodically.
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [47]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()

        output = model(data)
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [48]:
for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296016
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.281802
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.253489
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.207014
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.211993
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.167615
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.109080
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.093056
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.019979
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.966872
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.890509
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.836447
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.733401
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.645925
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.626742
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.568094
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.373687
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.311444
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.284683
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.109026
Train Epoch: 1 [200

## 2. Tasks

### 1. Please use other activation functions, e.g., sigmoid, tanh, and then plot the training loss and testing accuracy.

When plotting the training loss, the x-axis is iteration and the y-axis is training loss. When plotting the testing accuracy,  the x-axis is epoch and the y-axis is the training loss.

In [49]:
# your code
# build an mlp


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)   # linear layer (784 -> 256)
        self.fc2 = nn.Linear(256,128)  # linear layer (256 -> 128)
        self.fc3 = nn.Linear(128,64)  # linear layer (128 -> 10)
        self.fc4 = nn.Linear(64,32)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,10)

    def forward(self, x):
        h0 = x.view(-1,28*28) #input layer
        h1 = F.relu(self.fc1(h0)) # hidden layer 1
        h2 = F.relu(self.fc2(h1)) # hidden layer 2
        h3 = F.sigmoid(self.fc3(h2))
        h4 = F.tanh(self.fc4(h3))
        h5 = F.tanh(self.fc5(h4))
        h6 = self.fc6(h5) # output layer

        return h6

model = Net()
model.cuda() # put the model on GPU
print(model)

test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = data.cuda(), target.cuda()

    output = model(data)
    test_loss += criterion(output, target).item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))






Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=16, bias=True)
  (fc6): Linear(in_features=16, out_features=10, bias=True)
)

Test set: Average loss: 0.0002, Accuracy: 14/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 28/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 41/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 57/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 67/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 73/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 84/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 99/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 110/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 126/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 140

### 2. Please use different layers in the model, e.g., 1 layer, 5 layers, 10 layers,  and then plot the training loss and testing accuracy.

In [53]:
# your code
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)   # linear layer (784 -> 256)
        self.fc6 = nn.Linear(256,10)

    def forward(self, x):
        h0 = x.view(-1,28*28) #input layer
        h1 = F.relu(self.fc1(h0)) # hidden layer 1
        h6 = self.fc6(h1) # output layer

        return h6

model = Net()
model.cuda() # put the model on GPU
print(model)

test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = data.cuda(), target.cuda()

    output = model(data)
    test_loss += criterion(output, target).item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc6): Linear(in_features=256, out_features=10, bias=True)
)

Test set: Average loss: 0.0002, Accuracy: 11/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 19/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 26/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 36/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 45/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 52/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 57/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 70/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 76/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 92/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 101/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 109/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 117/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 127/10000 (1%)


Test set: Average loss: 0.0002, Accuracy: 133/10000 (

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)   # linear layer (784 -> 256)
        self.fc2 = nn.Linear(256,128)  # linear layer (256 -> 128)
        self.fc3 = nn.Linear(128,64)  # linear layer (128 -> 10)
        self.fc4 = nn.Linear(64,32)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,10)

    def forward(self, x):
        h0 = x.view(-1,28*28) #input layer
        h1 = F.relu(self.fc1(h0)) # hidden layer 1
        h2 = F.relu(self.fc2(h1)) # hidden layer 2
        h3 = F.sigmoid(self.fc3(h2))
        h4 = F.tanh(self.fc4(h3))
        h5 = F.tanh(self.fc5(h4))
        h6 = self.fc6(h5) # output layer

        return h6

model = Net()
model.cuda() # put the model on GPU
print(model)

test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = data.cuda(), target.cuda()

    output = model(data)
    test_loss += criterion(output, target).item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(28*28, 256)   # linear layer (784 -> 256)
        self.fc2 = nn.Linear(256,128)  # linear layer (256 -> 128)
        self.fc3 = nn.Linear(128,64)  # linear layer (128 -> 10)
        self.fc4 = nn.Linear(64,32)
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,8)
        self.fc7 = nn.Linear(8,4)
        self.fc8 = nn.Linear(4,16)
        self.fc9 = nn.Linear(16,8)
        self.fc10 = nn.Linear(8,4)
        self.fc11 = nn.Linear(4,10)

    def forward(self, x):
        h0 = x.view(-1,28*28) #input layer
        h1 = F.relu(self.fc1(h0)) # hidden layer 1
        h2 = F.relu(self.fc2(h1)) # hidden layer 2
        h3 = F.sigmoid(self.fc3(h2))
        h4 = F.tanh(self.fc4(h3))
        h5 = F.tanh(self.fc5(h4))
        h6 = F.tanh(self.fc6(h5))
        h7 = F.tanh(self.fc7(h6))
        h8 = F.tanh(self.fc8(h7))
        h9 = F.tanh(self.fc9(h8))
        h10 = F.tanh(self.fc10(h9))
        h11 = self.fc11(h10) # output layer

        return h11

model = Net()
model.cuda() # put the model on GPU
print(model)

test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = data.cuda(), target.cuda()

    output = model(data)
    test_loss += criterion(output, target).item()
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=16, bias=True)
  (fc6): Linear(in_features=16, out_features=8, bias=True)
  (fc7): Linear(in_features=8, out_features=4, bias=True)
  (fc8): Linear(in_features=4, out_features=16, bias=True)
  (fc9): Linear(in_features=16, out_features=8, bias=True)
  (fc10): Linear(in_features=8, out_features=4, bias=True)
  (fc11): Linear(in_features=4, out_features=10, bias=True)
)

Test set: Average loss: 0.0002, Accuracy: 11/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 16/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 24/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 35/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 45/10000 (0%)


Test set: Average loss: 0.0002, Accuracy: 6